# CASE STUDY: PROMPT ENGINEERING FOR CYBER THREAT INTELLIGENCE

Modern Security Operations Centers (SOCs) rely on automated systems to ingest and interpret real-time vulnerability intelligence feeds.

However, Large Language Models (LLMs) frequently fall in:

- Semantic version comparison
- Interval boundary reasoning
- Corporate mitigation nuance interpretation
- Multi-document synthesis

This case study investigates whether Prompt Engineering alone can transform a naïve LLM into a
structured cyber threat reasoning assistant.

## Dataset

Contains 2018 CVE (Common Vulnerabilities and Exposures) data taken from National Vulnerability Database.
2018 is chosen due to the presence of vendor statements for the CVEs which may help in giving a better response.

The first task is of extraction:

- Load the entire JSON data. (It is huge with millions of lines of code!)
- You must implement an extraction script (extract_and_mask_target_cves) to isolate only the three target CVEs required for this lab.
- Your script must actively extract crucial IDs such as descriptions, configurations and vendorComments and remove fields which may not contribute to our analysis.
- Save this filtered, processed output as cve_dataset.json.
- Once your dataset is generated, you need to write a utility to search cve_dataset.json for the matching 3 CVE IDs  and load only that specific JSON object into a masked_cve_json variable. By doing this, you will be able to rapidly test your prompt architectures against all three vulnerabilities without rewriting your code.



In [ ]:
import json

def extract_and_mask_cves(input_filename, output_filename):
    pass
INPUT_FILE = "nvdcve-2.0-2018.json" 
OUTPUT_FILE = "cve_dataset.json"

extract_and_mask_cves(INPUT_FILE, OUTPUT_FILE)

# Models used in this lab

We will be using Groq API in this lab. Choose any model of your choice from - https://console.groq.com/docs/rate-limits

It gives multiple options with cloud-based models that are pretty good for free.

In [ ]:
!pip install groq

In [ ]:
import json
import os
from huggingface_hub import InferenceClient
from groq import Groq

# Students will put in .env or secrets 
client = Groq(
    api_key=""
)
# The extracted JSON dataset
with open("cve_dataset.json", "r") as file:
    data = json.load(file)

# The specific CVE entries we are taking for evaluation
target_cve_ids = [
    "CVE-2018-3810", 
    "CVE-2018-3814", 
    "CVE-2018-6523"
]

# Create a new dictionary to hold the extracted CVEs
masked_cve_json = {}

# Iterate through the list of vulnerabilities
for vuln in data.get("vulnerabilities", []):
    current_cve_id = vuln.get("cve", {}).get("id")
    
    # Check if this ID is in our target list
    if current_cve_id in target_cve_ids:
        masked_cve_json[current_cve_id] = vuln["cve"]

print("CVE JSON for Evaluation:")
print(json.dumps(masked_cve_json, indent=2))

## Zero shot 

Zero-shot prompting is an AI technique where models perform tasks without examples.

You have to design the system prompt in such a way that it evaluates the json data. 

The SYSTEM prompt must include:
- Extraction of internal asset profile.
- Base your decision based on required fields (you need to identify which is relevant and specify them)
- Output exactly three things: CLASSIFICATION: (SAFE, LOW, MEDIUM, or HIGHLY VULNERABLE), SCORE: (A risk score from 0 to 10)

and REASONING: (Explaining your verdict based on the JSON fields)

USER prompt must include the JSON data.

Tune the parameters according to the task at end.

For example: How many tokens should be in your response? (max_tokens)

How much randomness you want in your response? (temperature)

AND explore more of the params and give your reasoning behind those choices.

In [ ]:
def evaluate_zero_shot(cve_data):
    pass

In [ ]:
for cve_id, cve_data in masked_cve_json.items():
    print(f"Evaluating: {cve_id}")
    zero_shot_result = evaluate_zero_shot(cve_data)
    
    print("Zero-Shot Evaluation Result:")
    print(zero_shot_result)

## One-shot

One-shot prompting is a technique where a model is provided with a single example of a task before being asked to perform similar tasks. This approach is especially relevant for large language models (LLMs) and sits between zero-shot prompting (no examples) and few-shot prompting (multiple examples).



The only difference here would be giving an example in the SYSTEM prompt like:

-- EXAMPLE START --

                        JSON DATA: {"descriptions": [{"value": "SQL Injection in Oturia Plugin before 3.5 allows unauthenticated queries."}], "configurations": [{"cpeMatch": [{"criteria": "cpe:2.3:a:oturia:smart_google_code_inserter:*:*:*:*:*:wordpress:*:*", "versionEndExcluding": "3.5"}]}], "vendorComments": []}
                        OUTPUT:
                        CLASSIFICATION: HIGHLY VULNERABLE
                        SCORE: 9.1/10
                        REASONING: The 'configurations' field shows versions before 3.5 are vulnerable. The asset is v3.4.2, meaning it is mathematically within the vulnerable range. The 'descriptions' field indicates unauthenticated SQL queries (high impact). There is no 'vendorComments' field, meaning no mitigation exists.

-- EXAMPLE END --

We recommend to make your own example for better understanding.

In [ ]:
def evaluate_one_shot(cve_data):
    pass

In [ ]:
for cve_id, cve_data in masked_cve_json.items():
    print(f"Evaluating: {cve_id}")
    one_shot_result = evaluate_one_shot(cve_data)
    print("\nOne-Shot Evaluation Result:")
    print(one_shot_result)

## Few-shot

Few-shot prompting is a technique where a model is provided with multiple examples of a task before being asked to perform similar tasks. 

The only difference here would be giving multiple such examples in the SYSTEM prompt with  variations like:

-- EXAMPLE START --

                        JSON DATA: {"descriptions": [{"value": "SQL Injection in Oturia Plugin before 3.5 allows unauthenticated queries."}], "configurations": [{"cpeMatch": [{"criteria": "cpe:2.3:a:oturia:smart_google_code_inserter:*:*:*:*:*:wordpress:*:*", "versionEndExcluding": "3.5"}]}], "vendorComments": []}
                        OUTPUT:
                        CLASSIFICATION: HIGHLY VULNERABLE
                        SCORE: 9.1/10
                        REASONING: The 'configurations' field shows versions before 3.5 are vulnerable. The asset is v3.4.2, meaning it is mathematically within the vulnerable range. The 'descriptions' field indicates unauthenticated SQL queries (high impact). There is no 'vendorComments' field, meaning no mitigation exists.

-- EXAMPLE END --

-- EXAMPLE START --

                    JSON DATA: {"descriptions": [{"value": "In nProtect AVS V4.0 before 4.0.0.39, the driver file allows local users to cause a denial of service (BSOD)."}], "configurations": [{"cpeMatch": [{"criteria": "cpe:2.3:a:inca:nprotect_avs:*:*:*:*:*:*:*:*", "versionStartIncluding": "4.0", "versionEndExcluding": "4.0.0.39"}]}], "vendorComments": [{"comment": "The fixed version(V4.0.0.39) can be downloaded through the link below... [http://avsd.nprotect.net](http://avsd.nprotect.net)"}]}
                    OUTPUT:
                    CLASSIFICATION: HIGHLY VULNERABLE
                    SCORE: 8.2/10
                    REASONING: The 'configurations' field establishes a vulnerable boundary of >= 4.0 and < 4.0.0.39. The asset is running version 4.0.0.35, which falls mathematically inside the vulnerable range. The 'vendorComments' provide a direct internet download link for a patch. However, the Internal Asset Profile explicitly states this workstation is air-gapped with NO internet access. Therefore, the vendor mitigation is void, and the asset remains fully exposed to a Denial of Service attack.
-- EXAMPLE END --

-- EXAMPLE START --

                    JSON DATA: {"descriptions": [{"value": "In Octopus Deploy versions 3.2.11 - 4.1.5 (fixed in 4.1.6), an authenticated user with ProcessEdit permission could reference an Azure account to bypass scoping restrictions, resulting in a potential escalation of privileges."}], "configurations": [{"cpeMatch": [{"criteria": "cpe:2.3:a:octopus:octopus_deploy:*:*:*:*:*:*:*:*", "versionStartIncluding": "3.2.11", "versionEndIncluding": "4.1.5"}]}], "vendorComments": []}
                    OUTPUT:
                    CLASSIFICATION: SAFE
                    SCORE: 0.0/10
                    REASONING: The 'configurations' field strictly bounds the vulnerability between versions 3.2.11 and 4.1.5 (inclusive). The internal asset is currently running version 4.1.8. Mathematically, 4.1.8 > 4.1.5, placing the asset completely outside the affected interval boundaries. Regardless of the 'descriptions' detailing privilege escalation, the software version is not susceptible to this specific CVE. 
-- EXAMPLE END --

In [ ]:
def evaluate_few_shot(cve_data):
    pass

In [ ]:
for cve_id, cve_data in masked_cve_json.items():
    print(f"Evaluating: {cve_id}")
    few_shot_result = evaluate_few_shot(cve_data)
    print("\nFew-Shot Evaluation Result:")
    print(few_shot_result)

## Chain of Thought (CoT) Prompting

Chain of thought (CoT) is a prompt engineering technique that enhances the output of large language models (LLMs), particularly for complex tasks involving multistep reasoning. It facilitates problem-solving by guiding the model through a step-by-step reasoning process by using a coherent series of logical steps. 


You must include the required steps involved in generating the response. (IN THE SYSTEM PROMPT)

And then mention it as 

Step 1: ...

Step 2: .. 
 
and so on. (FINAL STEP BEING WHAT DO YOU WANT AS OUTPUT - CLASSIFICATION, SCORE, REASONING)


In [ ]:
import json
import os
from huggingface_hub import InferenceClient

def evaluate_cot(cve_data):
    pass

In [ ]:
for cve_id, cve_data in masked_cve_json.items():
    print(f"Evaluating: {cve_id}")
    cot_result = evaluate_cot(cve_data)
    print("\nChain-of-Thought Evaluation Result:")
    print(cot_result)


## Tree of Thought (ToT)

Tree of Thought (ToT) prompting is a sophisticated prompt engineering technique for large language models (LLMs) that structures the reasoning process as a branching tree, enabling the model to explore, evaluate and refine multiple reasoning paths in parallel. This approach is designed to mimic human problem solving, where several options are considered, intermediate steps are evaluated and less promising directions are abandoned in favor of more fruitful ones.

You must try this only for CVEs with vendorComments.

One of the steps must include looking at different branches:

- Branch A (Optimistic): Assume the 'vendorComments' (if present) provide a valid workaround that makes our specific asset safe. Calculate a hypothetical Score (1-4).
- Branch B (Cynical): Assume the 'vendorComments' (if present) are just PR deflection or the workaround fails because of our specific Internal Asset Network Status. Calculate a hypothetical Score (8-10).
- Branch C (Neutral): Assume the vulnerability requires complex local access, making the 'descriptions' impact moderate. Calculate a hypothetical Score (5-7).


In [ ]:
import json
import os
from huggingface_hub import InferenceClient

def evaluate_tot(cve_data):
    pass

In [ ]:
tot_result = evaluate_tot(masked_cve_json)
print("\nTree-of-Thought Evaluation Result:")
print(tot_result)

## Graph of Thought (GoT)

Graph of Thought (GoT) Prompting is a framework that enhances large language models' (LLMs) reasoning capabilities by modeling thoughts as a dynamic, interconnected graph rather than a linear or tree-like structure.  Developed by researchers at ETH Zürich and published in August 2023, GoT represents LLM-generated ideas as vertices (nodes) and their relationships as edges.

You will need to mention the nodes and edges to look at different relationships within the input data, in the steps of the system prompt.


- Node 1 (CVE Threat): Extract the core threat from 'descriptions' and boundaries from 'configurations'. Estimate a Base Impact Score (1-10) based on the severity of the description.
- Node 2 (Vendor Status): Extract mitigation status from 'vendorComments' (if present). Assign a Mitigation Modifier (e.g., -2 or -3 for a patch/workaround, 0 if no comment).
- Node 3 (Internal Asset): State the extracted asset software/version and its Environmental Constraints (e.g., No Internet).

- Edge A (Version Match): Does the Internal Asset (Node 3) fall inside the vulnerable bounds of the Threat (Node 1)? [True/False]. If False, Final Score automatically becomes 0.
- Edge B (Mitigation Applicability): Can the mitigation in Node 2 be successfully applied given the environment constraints in Node 3? [True/False]. If False (e.g., requires a download but asset has no internet), the Mitigation Modifier from Node 2 is nullified (becomes 0).

Create an output table of the format:
| Asset | CVE | Vulnerable | Vendor Status | Action |


In [ ]:
# ==========================================
# PHASE 4: GRAPH-OF-THOUGHT (GoT)
# ==========================================
import json

def evaluate_got(cve_data):
    pass

In [ ]:
for cve_id, cve_data in masked_cve_json.items():
    print(f"Evaluating: {cve_id}")
    got_result = evaluate_got(cve_data)
    print("\n=== Phase 4: Graph-of-Thought Evaluation Result ===")
    print(got_result)


## ReAct (Reason + Act) LOOP

ReAct (Reasoning + Acting) Prompting is a technique used to improve AI models in solving problems. It combines two important processes i.e reasoning (thinking through the problem) and acting (taking actions based on that thinking). 


Thought -> Action -> Observation Loop. You can keep max_iterations = 6.

In [ ]:
# ==========================================
# PHASE 5: ReAct (Reason + Act) LOOP
# ==========================================
import json

def evaluate_react(cve_data, max_steps=6):
    pass

In [ ]:
evaluate_react(masked_cve_json)

## EXPLORATORY PROMPT ENGINEERING

#### Technique 1: Self-Consistency Prompting (Majority Voting)

This aims "to replace the naive greedy decoding used in chain-of-thought prompting". 

The idea is to sample multiple, diverse reasoning paths through few-shot CoT, and use the generations to select the most consistent answer. 

This helps to boost the performance of CoT prompting on tasks involving arithmetic and commonsense reasoning.

#### Technique 2: Prompt Decomposition (Micro-Prompts)

Divide and conquer approach in prompting.

#### Technique 3: Role-Based Prompting

To better understand the context behind the domain of the questions.

#### Technique 4: Reflection Prompting

Reflection Prompting involves prompting the AI to analyze its own response before finalizing it. Instead of just generating an answer, the AI is guided to review its initial output, identify potential weaknesses, and suggest improvements. It’s similar to how a writer edits their own draft, playing both the role of the creator and the reviewer, resulting in higher-quality and more reliable outputs.


In [ ]:
# ==========================================
# PHASE 6: EXPLORATORY PROMPT ENGINEERING
# ==========================================
import json
import re
from collections import Counter

# Standardized environmental constraint for all Phase 6 tests
environmental_constraint = "INTERNAL ASSET PROFILE: Highly sensitive local workstation (NO direct internet access)."

# ---------------------------------------------------------
# Technique 1: Self-Consistency Prompting (Majority Voting)
# ---------------------------------------------------------
def evaluate_self_consistency(cve_data, num_chains=5):
    pass
# ---------------------------------------------------------
# Technique 2: Prompt Decomposition (Micro-Prompts)
# ---------------------------------------------------------
def evaluate_prompt_decomposition(cve_data):
    pass

# ---------------------------------------------------------
# Technique 3: Role-Based Prompting
# ---------------------------------------------------------
def evaluate_role_based(cve_data):
    pass

# ---------------------------------------------------------
# Technique 4: Reflection Prompting
# ---------------------------------------------------------
def evaluate_reflection(cve_data):
    pass

In [ ]:
# ==========================================
# Execute Phase 6 (choose any 3)
# ==========================================
evaluate_self_consistency(masked_cve_json)
evaluate_prompt_decomposition(masked_cve_json)
evaluate_role_based(masked_cve_json)
evaluate_reflection(masked_cve_json)

# REFERENCES

1. https://nvd.nist.gov
2. https://www.redhat.com/en/topics/security/what-is-cve
3. https://www.ibm.com/think/topics/cve
4. https://console.groq.com/docs/models
5. https://console.groq.com/docs/rate-limits
6. https://console.groq.com/docs/quickstart
7. Chain-of-Thought (CoT): Wei, J., et al. (2022). Chain-of-Thought Prompting Elicits Reasoning in Large Language Models. arXiv. https://arxiv.org/abs/2201.11903

8. Tree-of-Thought (ToT): Yao, S., et al. (2023). Tree of Thoughts: Deliberate Problem Solving with Large Language Models. arXiv. https://arxiv.org/abs/2305.10601

9. Graph-of-Thought (GoT): Besta, M., et al. (2023). Graph of Thoughts: Solving Elaborate Problems with Large Language Models. arXiv. https://arxiv.org/abs/2308.09687

10. ReAct (Reason + Act): Yao, S., et al. (2022). ReAct: Synergizing Reasoning and Acting in Language Models. arXiv. https://arxiv.org/abs/2210.03629

11. Self-Consistency: Wang, X., et al. (2022). Self-Consistency Improves Chain of Thought Reasoning in Language Models. arXiv. https://arxiv.org/abs/2203.11171

12. Reflection / Reflexion: Shinn, N., et al. (2023). Reflexion: Language Agents with Verbal Reinforcement Learning. arXiv. https://arxiv.org/abs/2303.11366

13. Common Vulnerability Scoring System (CVSS): Official specification by FIRST (Forum of Incident Response and Security Teams) detailing how base and environmental severity scores are calculated. https://www.first.org/cvss/

14. Semantic Versioning (SemVer): The official specification for Major.Minor.Patch software versioning. https://semver.org/